In [ ]:
# this is the required libraries
!pip install transformers datasets --quiet

# Upload the new big dataset file
from google.colab import files
uploaded = files.upload()  # upload long_stories_40k.txt here

# Import necessary modules
from datasets import Dataset
from transformers import GPT2Tokenizer

# Loading the new large text data
file_path = "long_stories_40k.txt"
with open(file_path, "r", encoding="utf-8") as f:
    lines = [line.strip() for line in f if line.strip()]

# Convert to the Hugging Face Dataset
dataset_dict = {"text": lines}
dataset = Dataset.from_dict(dataset_dict)

# Tokenization
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Set pad token

def tokenize(example):
    return tokenizer(example["text"], truncation=True, max_length=128)

tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns=["text"])

# Load GPT-2 model and data collator
from transformers import GPT2LMHeadModel, DataCollatorForLanguageModeling

model = GPT2LMHeadModel.from_pretrained("gpt2")
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Training configuration
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./gpt2-long-stories-finetuned",
    num_train_epochs=2,                  # You can adjust epochs here
    per_device_train_batch_size=4,       # Adjust based on GPU memory
    save_steps=500,
    logging_steps=100,
    save_total_limit=1,
    prediction_loss_only=True,
    report_to="none"
)

# Start training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)




Saving long_stories_40k.txt to long_stories_40k (2).txt


Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,1.498900
200,0.394100
300,0.218300
400,0.196800
500,0.190800
600,0.180200
700,0.175000
800,0.164500
900,0.166500
1000,0.164500


TrainOutput(global_step=20000, training_loss=0.15929956302642823, metrics={'train_runtime': 2474.0953, 'train_samples_per_second': 32.335, 'train_steps_per_second': 8.084, 'total_flos': 1769373697536000.0, 'train_loss': 0.15929956302642823, 'epoch': 2.0})

In [ ]:
# Save the fine-tuned model and tokenizer
model.save_pretrained("/content/gpt2-long-stories-final")
tokenizer.save_pretrained("/content/gpt2-long-stories-final")

('/content/gpt2-long-stories-final/tokenizer_config.json',
 '/content/gpt2-long-stories-final/special_tokens_map.json',
 '/content/gpt2-long-stories-final/vocab.json',
 '/content/gpt2-long-stories-final/merges.txt',
 '/content/gpt2-long-stories-final/added_tokens.json')

In [ ]:


# Reload the fine-tuned model for generation
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_path = "/content/gpt2-long-stories-final"
model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2Tokenizer.from_pretrained(model_path)

# Function to generate story/quote
def generate_quote(prompt, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        **inputs,
        do_sample=True,
        max_length=max_length,
        top_k=50,
        top_p=0.9,
        temperature=0.9,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# (Optional) You can customize or remove this category detector or modify it for story prompts
def detect_category(user_input):
    text = user_input.lower()
    if any(word in text for word in ["adventure", "journey", "quest", "explore"]):
        return "[ADVENTURE]"
    elif any(word in text for word in ["fantasy", "magic", "dragon", "wizard"]):
        return "[FANTASY]"
    elif any(word in text for word in ["space", "future", "robot", "alien"]):
        return "[SCI-FI]"
    elif any(word in text for word in ["horror", "ghost", "fear", "darkness"]):
        return "[HORROR]"
    else:
        return "[GENERAL]"

# User prompt input
print("Enter your story prompt (e.g. adventure in the mountains, magic castle, alien invasion, haunted house):")
user_input = input("Your prompt: ")

# Add category tag automatically
category_token = detect_category(user_input)
final_prompt = f"{category_token} {user_input}"

# Generate and print the result
result = generate_quote(final_prompt)
print("\nGenerated Story:")
print(result)

# Compress the model folder into a zip file
from shutil import make_archive
make_archive("/content/gpt2-long-stories-final", 'zip', "/content/gpt2-long-stories-final")


Enter your story prompt (e.g. adventure in the mountains, magic castle, alien invasion, haunted house):
Your prompt: magic 

Generated Story:
[FANTASY] magic  cursed knight sought redemption by questing for a legendary sword said to break all enchantments and restore his lost honor. Danger lurked in every shadow, but they pressed onward. Few dared to follow where they had brave


'/content/gpt2-long-stories-final.zip'